## スクレイピング

In [25]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import unicodedata 

In [26]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import unicodedata 

# 全角半角に変換する関数
def normalize_text(text):
    return unicodedata.normalize("NFKC", text)

driver = webdriver.Chrome()

# 指定されたURLにアクセス
url = "https://www.jalan.net/uw/uwp2011/uww2011init.do?keyword=%83f%83B%83Y%83j%81%5B&distCd=06&rootCd=7701&screenId=FWPCTOP&ccnt=button-fw&image1="
driver.get(url)

hotels_data = []

while True:
    # ページのHTMLコードを取得
    html_code = driver.page_source
    soup = BeautifulSoup(html_code, "html.parser")

    # 必要な情報を抽出
    hotel_items = soup.find_all('li', class_='p-yadoCassette p-searchResultItem')

    for hotel in hotel_items:
        
        # ホテル名
        hotel_name = normalize_text(hotel.find('h2', class_='p-searchResultItem__facilityName').text.strip())

        # 住所
        address_tag = hotel.find('dt', text="住所")
        address = normalize_text(address_tag.find_next('dd').text.strip() if address_tag else None)

        # アクセス
        access_tag = hotel.find('dt', text="アクセス")
        access = normalize_text(access_tag.find_next('dd').text.strip() if access_tag else None)

        # 料金
        price = normalize_text(hotel.find('span', class_='p-searchResultItem__lowestPriceValue').text.strip())

        # 結果を表示
        print(f"ホテル名: {hotel_name}")
        print(f"料金: {price}")
        print(f"住所: {address}")
        print(f"アクセス: {access}")
        print("-" * 40)

        # データを保存
        hotels_data.append(f'(\n    "{hotel_name}",  \n    "{price}",  \n    "{address}",  \n    "{access}"  \n),')

    # 次のページに進む処理
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
        next_button.click()
        time.sleep(2)
    except NoSuchElementException:
        break

driver.quit()

# 出力を保存
output_file = "hotels_data_.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write("[\n")
    file.write("\n".join(hotels_data))
    file.write("\n]")


### DB作成

In [27]:
import sqlite3

In [28]:
!pwd

/Users/oo/DSp


In [29]:
# DBファイルを保存するファイルパス
path = '/Users/oo/DSp'

# ローカル
# 絶対パスで指定しておく（仮想環境作成している場合は，相対パスで指定）
# path = ''

# DBファイル名
db_name = 'tripdata.db'

# DBに接続する（DBファイルが存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
con.close()

### テーブル作成

In [38]:
# DBに接続
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# 実行したいSQL
sql = 'CREATE TABLE hotels (id INT AUTO_INCREMENT PRIMARY KEY,  hotel_name VARCHAR(255) NOT NULL,price VARCHAR(50),address VARCHAR(255),access TEXT,created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP );'

# SQLを実行
cur.execute(sql)

# DBへの接続を閉じる
con.close()

OperationalError: table hotels already exists

### スクレイピングしたデータをDBに挿入・保存

In [39]:
# DBに接続
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを挿入するSQL
sql_insert_many = """
INSERT INTO hotels (hotel_name, price, address, access) 
VALUES (?, ?, ?, ?);
"""

# スクレイピングで取得したデータ
hotels_data = [
    (
    "シェラトン・グランデ・トーキョーベイ・ホテル",  
    "8,700円~/人",  
    "千葉県浦安市舞浜1―9",  
    "電車/ディズニーリゾートラインベイサイド・ステーション徒歩1分 車/首都高速湾岸線 葛西又は浦安出口"  
),
(
    "ホテルオークラ東京ベイ",  
    "8,008円~/人",  
    "千葉県浦安市舞浜1-8",  
    "JR舞浜駅下車。ディズニーリゾートライン2駅目ベイサイド・ステーションから送迎バスまたは徒歩約3分。"  
),
(
    "東京ディズニーシー・ホテルミラコスタ(R)",  
    "45,950円~/人",  
    "千葉県浦安市舞浜1-13",  
    "JR京葉線・武蔵野線舞浜駅下車。ディズニーリゾートライン「東京ディズニーシー・ステーション」下車。"  
),
(
    "東京ベイ舞浜ホテル",  
    "11,827円~/人",  
    "千葉県浦安市舞浜1-34",  
    "無料シャトルバスにてJR舞浜駅より約5分、またはベイサイド・ステーションより約4分"  
),
(
    "ディズニーアンバサダー(R)ホテル",  
    "24,750円~/人",  
    "千葉県浦安市舞浜2-11",  
    "東京駅から JR京葉線・武蔵野線で約15分「舞浜駅(南口)」下車、徒歩8分"  
),
(
    "東京ディズニーランド(R)ホテル",  
    "31,750円~/人",  
    "千葉県浦安市舞浜29-1",  
    "JR京葉線・武蔵野線「舞浜駅(南口)」下車、徒歩8分"  
),
(
    "東京ディズニーセレブレーションホテル(R)",  
    "11,500円~/人",  
    "千葉県浦安市明海7-1-1",  
    "JR京葉線/武蔵野線舞浜駅下車→徒歩5分→東京ディズニーランド(R)より無料シャトルバスにて約20分"  
),
(
    "東京ディズニーリゾート・トイ・ストーリー(R)ホテル",  
    "27,200円~/人",  
    "千葉県浦安市舞浜1-47",  
    "JR京葉線・武蔵野線「舞浜駅」乗換、ディズニーリゾートライン 「ベイサイド・ステーション」下車徒歩約3分"  
),
(
    "イビススタイルズ東京ベイ",  
    "6,075円~/人",  
    "千葉県浦安市日の出2-1-1",  
    "JR京葉・武蔵野線で東京駅~新浦安駅間約17分 新浦安駅から無料シャトルバス タクシー約5分"  
),
(
    "オリエンタルホテル東京ベイ",  
    "9,900円~/人",  
    "千葉県浦安市美浜1-8-2",  
    "JR京葉線・武蔵野線 新浦安駅(舞浜駅まで1駅3分)直結 徒歩30秒、首都高速湾岸線 浦安ICより約5分"  
),
(
    "ヒルトン東京ベイ",  
    "13,639円~/人",  
    "千葉県浦安市舞浜1-8",  
    "JR京葉線・武蔵野線舞浜駅よりディズニーリゾートライン乗車、ベイサイド駅下車徒歩1分。"  
),
(
    "東京ベイ舞浜ホテル ファーストリゾート",  
    "6,696円~/人",  
    "千葉県浦安市舞浜1-6",  
    "JR舞浜駅より無料送迎バスで約5~10分、又はモノレール・ベイサイドステーション下車、送迎バス約6分。"  
),
(
    "浦安ブライトンホテル東京ベイ",  
    "7,330円~/人",  
    "千葉県浦安市美浜1丁目9",  
    "JR京葉線・武蔵野線新浦安駅直結。JR東京駅より快速で約17分。東京ディズニーリゾート(R)までバス直行。"  
),
(
    "ホテルエミオン東京ベイ",  
    "4,800円~/人",  
    "千葉県浦安市日の出1-1-1",  
    "JR京葉線新浦安駅下車徒歩7分。首都高速湾岸線浦安出口より5分。パークへ無料シャトルバスで約15分"  
),
(
    "三井ガーデンホテルプラナ東京ベイ",  
    "8,000円~/人",  
    "千葉県浦安市明海6-2-1",  
    "三井ガーデンホテルプラナ東京ベイは、東京ディズニーリゾート(R)・パートナーホテルです。大浴場が人気!"  
),
(
    "ラ・ジェント・ホテル東京ベイ",  
    "3,850円~/人",  
    "千葉県浦安市日の出5-7-1",  
    "シャトルバスの時刻表はホテル公式HPに掲載しております。"  
),
(
    "きらくホテル",  
    "6,050円~/人",  
    "千葉県千葉市稲毛区稲毛1-14-3",  
    "JR総武線稲毛駅・京葉線稲毛海岸駅 徒歩15分 タクシー5分。 京成稲毛駅 徒歩5分。"  
),
(
    "ELEhotelGinzaEast",  
    "5,400円~/人",  
    "東京都中央区新富1-15-10",  
    "東京メトロ・JR 八丁堀駅より徒歩にて約5分★東京メトロ 新富町駅より徒歩にて約5分"  
),
(
    "ホテルがんけ",  
    "5,680円~/人",  
    "茨城県鹿嶋市宮中1-3-6",  
    "鹿島神宮駅から徒歩3分/潮来I.Cから車で約10分"  
),
(
    "ホテル ルミエール西葛西",  
    "4,100円~/人",  
    "東京都江戸川区西葛西6-20-3",  
    "東京メトロ東西線「西葛西」下車 南口より徒歩5分"  
),
(
    "【変なホテル東京西葛西】シャトルバスで舞浜方面へ楽々アクセス",  
    "5,460円~/人",  
    "東京都江戸川区西葛西5-4-7",  
    "ビジネス利用にも便利な東京メトロ東西線・西葛西駅徒歩3分・舞浜駅までの無料シャトルあり"  
),
(
    "コンフォートスイーツ東京ベイ",  
    "7,000円~/人",  
    "千葉県浦安市明海5丁目8番15号",  
    "東京ディズニーリゾート(R)より無料シャトルバスで約25分◆「新浦安」駅南口より車(タクシー)で約10分"  
),
(
    "CVS・BAY HOTEL 本館(CVS・ベイホテル本館)",  
    "5,500円~/人",  
    "千葉県市川市塩浜2-33-1",  
    "JR市川塩浜駅、徒歩1分。首都高速湾岸線下り浦安ICより車10分。東関東自動車道湾岸市川ICより車10分"  
),
(
    "グランドニッコー東京ベイ 舞浜",  
    "11,000円~/人",  
    "千葉県浦安市舞浜1-7",  
    "舞浜駅よりディズニーリゾートラインにて2駅目「ベイサイド・ステーション」下車後、徒歩にて約4分"  
),
(
    "京王プラザホテル",  
    "40,700円~/人",  
    "東京都新宿区西新宿2-2-1",  
    "JR新宿駅(西口)下車徒歩5分、都営大江戸線都庁前駅B1出口すぐ。首都高速新宿ランプより約1キロ。"  
),
(
    "第一ホテル両国",  
    "5,500円~/人",  
    "東京都墨田区横網1-6-1",  
    "都営地下鉄大江戸線/両国駅下車A1出口直結。JR総武線/両国駅下車、徒歩約7分。"  
),
(
    "ホテルニューオータニ幕張",  
    "5,600円~/人",  
    "千葉県千葉市美浜区ひび野2-120-3",  
    "JR海浜幕張駅 徒歩5分 /幕張メッセ直結 / 成田空港・羽田空港へも好アクセス"  
),
(
    "CVS・BAY HOTEL 新館(CVS・ベイホテル新館)",  
    "4,200円~/人",  
    "千葉県市川市塩浜2-3-8",  
    "JR市川塩浜駅、徒歩3分。首都高速湾岸線下り浦安ICより車10分。東関東自動車道湾岸市川ICより車10分"  
),
(
    "ホテル柳橋",  
    "8,500円~/人",  
    "東京都台東区柳橋1-3-12",  
    "JR総武線浅草橋駅東口/都営浅草線浅草橋駅A1出口より徒歩2分"  
),
(
    "住庄ほてる",  
    "4,800円~/人",  
    "東京都中央区日本橋小舟町9-14",  
    "地下鉄日比谷線・浅草線人形町駅から徒歩3分、銀座線・半蔵門線三越前駅より徒歩7分 JR東京駅から車5分"  
),
(
    "両国リバーホテル",  
    "7,800円~/人",  
    "東京都墨田区両国2-13-8 両国リバーホテル内",  
    "電車/JR両国駅西口から徒歩1分 大江戸線都営両国駅A4出口より徒歩7分"  
),
(
    "錦の館",  
    "33,600円~/室",  
    "東京都江東区毛利2-8-7",  
    "錦糸町駅徒歩10分、地下鉄メトロ半蔵門線住吉駅徒歩7分"  
),
(
    "新習志野駅前 天然温泉 湯~ねる",  
    "4,000円~/人",  
    "千葉県習志野市茜浜2-2-1",  
    "JR新習志野駅 徒歩1分♪ ミスターマックス新習志野SC2階は入口です!"  
),
(
    "ビジネスホテル 福田屋",  
    "3,000円~/室",  
    "東京都台東区清川1-35-11",  
    "JR・東京メトロ日比谷線南千住駅下車徒歩13分。浅草や上野駅から都バスをご利用頂くとさらに便利です。"  
),
(
    "ビジネスホテル レガート",  
    "7,000円~/人",  
    "東京都江東区森下3-17-12",  
    "新宿線・菊川駅徒歩6分、大江戸線・森下駅徒歩8分、半蔵門線・清澄白河駅徒歩11分"  
),
(
    "THE BONDS HOTEL TOKYO",  
    "15,000円~/人",  
    "東京都墨田区向島3-43-4",  
    "東武スカイツリーライン/とうきょうスカイツリー駅から徒歩2分。浅草線/押上駅から徒歩5分。"  
),
(
    "アーバンステイ品川",  
    "7,290円~/人",  
    "東京都品川区北品川2-23-14",  
    "京浜急行「新馬場駅」から徒歩3分、各線「品川駅」から徒歩15分、臨海線「天王洲アイル駅」から徒歩12分。"  
),
(
    "星野リゾート 1955 東京ベイ",  
    "8,800円~/人",  
    "千葉県浦安市日の出7-2-3",  
    "新浦安駅より無料シャトルバスにて約10分、東京ディズニーリゾート(R)より無料シャトルバスにて約30分"  
),
(
    "アパホテル〈京急蒲田駅前西〉",  
    "4,000円~/人",  
    "東京都大田区蒲田4-18-24",  
    "京急線「京急蒲田駅」西口より徒歩2分。JR京浜東北線・東急線「蒲田駅」東口より徒歩8分。"  
),
(
    "ホテル堀留ヴィラ",  
    "7,000円~/人",  
    "東京都中央区日本橋堀留町1-10-10",  
    "地下鉄小伝馬町駅徒歩3分、地下鉄人形町駅徒歩4分"  
),
(
    "地方職員共済組合 プラザ菜の花",  
    "4,000円~/人",  
    "千葉県千葉市中央区長洲1-8-1",  
    "JR外房線本千葉駅下車徒歩3分。千葉都市モノレール県庁前駅隣接。"  
),
(
    "KIN HOTEL",  
    "11,150円~/人",  
    "東京都台東区松が谷2-3-2",  
    "東京メトロ銀座線稲荷町駅より徒歩にて約7分。全室個別空調、窓の開け閉めが可能、完璧な換気ができる!"  
),
(
    "小さなお宿 プチバスケット",  
    "5,500円~/人",  
    "千葉県千葉市中央区院内1-11-3",  
    "JR千葉駅から徒歩15分 JR東千葉駅から徒歩10分 京成千葉駅から徒歩15分 千葉モノレール栄町駅から徒歩6分"  
),
(
    "灯光旅館-高砂",  
    "13,500円~/人",  
    "東京都葛飾区高砂3-25-15",  
    "京成電鉄・都営浅草線高砂駅から徒歩5-7分"  
),
(
    "アーバンステイ押上",  
    "5,790円~/人",  
    "東京都墨田区文花1-10-1",  
    "各線「押上駅/とうきょうスカイツリー駅」から徒歩9分"  
),
(
    "勝浦一棟貸し_海の香",  
    "6,100円~/人",  
    "千葉県勝浦市松部1615-4",  
    "1JR外房線:鵜原駅より徒歩15分 2アクアライン市原鶴舞ICを降りて、国道297号経由にて約40分"  
),
(
    "ホテルプラザオーサカ",  
    "7,300円~/人",  
    "大阪府大阪市淀川区新北野1-9-15",  
    "阪急十三駅西口、徒歩5分。大阪駅から車で5分。新大阪駅から車で10分。"  
),
(
    "カプセル&サウナロスコ(旧カプセルイン駒込)",  
    "5,760円~/室",  
    "東京都北区中里2-4-8",  
    "JR山手線駒込駅東口より徒歩30秒 地下鉄南北線駒込駅ご利用の方もまずは山手線駒込駅東口を目指して下さい"  
),
(
    "ビースティーレ 舞浜",  
    "22,000円~/人",  
    "千葉県浦安市舞浜2-46-6",  
    "JR舞浜駅北口改札口歩道橋を正面の住宅地方面へ、突き当たりを左に下り小道で住宅地の中に入り左側すぐ。"  
),
(
    "チサンイン蒲田",  
    "4,050円~/人",  
    "東京都大田区蒲田4-23-13",  
    "京急蒲田駅西口より徒歩2分、JR蒲田駅東口より徒歩7分。羽田空港まで乗り換え無し、急行で11分。"  
),
(
    "ホテルニュータウン秋葉原",  
    "3,900円~/人",  
    "東京都千代田区神田佐久間町2-12-8",  
    "JR・日比谷線秋葉原駅 昭和通り改札口より信号を渡って三菱UFJ銀行の線路沿いから歩いて1分!"  
),
(
    "Tabist ホテルダイアナ 八千代台",  
    "4,150円~/人",  
    "千葉県八千代市八千代台南1-4-3",  
    "京成「八千代台」駅下車、改札を背に左手へ。東口ロータリー近くの「三菱東京UFJ銀行」さんの2件裏です。"  
),
(
    "Hotaku HOTEL",  
    "12,200円~/室",  
    "東京都千代田区岩本町二丁目13番1号",  
    "☆神田駅より徒歩7分☆秋葉原駅より徒歩7分☆岩本町駅より徒歩2分☆"  
),
(
    "東横INN品川大井町",  
    "9,975円~/人",  
    "東京都品川区大井1-27-3",  
    "JR京浜東北線・りんかい線・東急大井町線より徒歩5分。"  
),
(
    "Araiya",  
    "32,100円~/人",  
    "東京都港区高輪2-17-1",  
    "品川駅から1駅泉岳寺駅から徒歩0分。羽田・成田空港、ディズニーランドへのアクセス良好"  
),
(
    "アパホテル<TKP日暮里駅前>",  
    "5,000円~/人",  
    "東京都荒川区東日暮里5-52-9",  
    "JR山手・京成線「 日暮里 」駅南改札口から徒歩2分。上野駅3分・秋葉原駅7分・浅草駅まで14分とアクセス◎"  
),
(
    "カプセルホテル レインボー総武線・市川・本八幡店",  
    "3,200円~/室",  
    "千葉県市川市南八幡4-1-12 パインズビル2F",  
    "JR総武線本八幡駅【南口】 京成本八幡駅 都営新宿線本八幡駅"  
),
(
    "カプセルホテル レインボー総武線・葛飾区・新小岩店",  
    "3,700円~/人",  
    "東京都葛飾区新小岩1―49-1",  
    "JR新小岩駅南口 徒歩1分"  
),
(
    "三井ガーデンホテル京橋",  
    "16,300円~/人",  
    "東京都中央区京橋1丁目3番地6号",  
    "JR東京駅 八重洲中央口より徒歩5分、JR「東京」駅八重洲地下街24番出入口徒歩1分"  
),
(
    "FIRST CABIN(ファーストキャビン)愛宕山",  
    "6,550円~/人",  
    "東京都港区西新橋3丁目10番7号",  
    "新橋駅 烏森口 徒歩10分、都営三田線 御成門駅 A5出口 徒歩5分、日比谷線 虎ノ門ヒルズ駅 B2出口 徒歩7分"  
),
(
    "クインテッサホテル千葉船橋",  
    "2,900円~/人",  
    "千葉県船橋市湊町2-5-18",  
    "【電車】船橋駅南口 より徒歩12分、京成船橋駅 より8分 【お車】船橋I.C より約10分、花輪I.C より15分"  
),
(
    "アパホテル<蒲田駅東>",  
    "4,000円~/人",  
    "東京都大田区蒲田5丁目31-5",  
    "羽田空港から京急空港線でわずか10分、京急蒲田駅・西口から徒歩5分、JR蒲田駅・東口から徒歩5分"  
),
(
    "M4 design hotel",  
    "14,800円~/室",  
    "千葉県浦安市東野2-25-25",  
    "舞浜よりタクシー・お車にて約10分。駐車場無料!"  
),
(
    "フレックステイイン新浦安",  
    "3,700円~/人",  
    "千葉県浦安市今川4-1-29",  
    "舞浜駅よりバス15分 サンコーポ西口下車 徒歩2分 または新浦安駅北口より徒歩15分"  
),
(
    "ゲストハウスCROSS BORDER(舞浜・浦安)",  
    "3,300円~/人",  
    "千葉県浦安市富士見1-10-24-1F",  
    "舞浜駅バス7分、浦安駅バス10分「堀江中学校前」徒歩1分"  
),
(
    "サウナ&カプセルホテルニュー小岩310",  
    "2,800円~/人",  
    "東京都江戸川区西小岩1丁目19番24号",  
    "総武線(各駅停車)小岩駅(北口)より徒歩3分"  
),
(
    "HOTEL R9 Premium 市川駅前",  
    "5,300円~/人",  
    "千葉県市川市市川1丁目8-6",  
    "JR市川駅北口より徒歩1分。"  
),
(
    "新松戸ステーションホテル",  
    "6,500円~/人",  
    "千葉県松戸市新松戸2-120",  
    "JR各駅停車常磐線・武蔵野線新松戸駅より徒歩2分。"  
),
(
    "かぢや旅館",  
    "6,350円~/人",  
    "千葉県富津市金谷3887",  
    "JR浜金谷駅から徒歩5分 東京湾フェリーから車で2分"  
),
(
    "7Rooms Hotel & Cafe",  
    "8,250円~/人",  
    "東京都江戸川区中葛西3-9-11",  
    "東京メトロ 葛西駅より徒歩7分。緑豊かな公園の前に位置するホテルです。"  
),
(
    "アーバンホテル森下",  
    "7,000円~/人",  
    "東京都江東区森下3-20-15",  
    "都営新宿線 菊川駅より徒歩3分、都営大江戸線 森下駅より徒歩8分"  
),
(
    "IKIDANE Cozy Hotel 羽田空港",  
    "6,750円~/人",  
    "東京都大田区羽田五丁目13番4号",  
    "京浜急行線「天空橋駅」A2出口より最短で徒歩3分・東京モノレール「天空橋駅」から徒歩約8分"  
),
(
    "灯光旅館",  
    "7,000円~/人",  
    "東京都葛飾区青戸1-6-6",  
    "京成電鉄・都営浅草線青砥駅から徒歩3分"  
),
(
    "ゲストハウス品川宿 (Guest House 品川宿)",  
    "5,050円~/人",  
    "東京都品川区北品川1-22-16",  
    "京急線北品川駅徒歩2分、JR品川駅徒歩10分。羽田空港直通。成田空港やディズニーランドへのアクセス良好。"  
),
(
    "ホテルVintage<東京新富町>",  
    "15,503円~/人",  
    "東京都中央区入船2-10-7",  
    "日比谷線築地駅/4番出口より徒歩8分。 有楽町線/7番出口より徒歩3分。"  
),
(
    "シーナと一平",  
    "15,500円~/人",  
    "東京都豊島区長崎2-12-4",  
    "池袋駅から徒歩20分。西武池袋線「椎名町」駅から徒歩4分。東京メトロ「要町」駅から徒歩12分程度。"  
),
(
    "お宿 きえん",  
    "6,500円~/人",  
    "東京都墨田区亀沢2丁目21-6",  
    "大江戸線両国駅徒歩8分 JR両国駅徒歩13分 首都高向島出口5分 浜町出口5分 錦糸町出口5分"  
),
(
    "東横INN東西線西葛西",  
    "8,400円~/人",  
    "東京都江戸川区西葛西5-11-12",  
    "東京メトロ東西線 西葛西 駅より徒歩2分"  
),
(
    "Bamba Hotel",  
    "17,300円~/人",  
    "東京都品川区南品川1-1-2",  
    "品川駅より2駅の京急線新馬場駅南口より徒歩0分、羽田空港、ディズニーへのアクセス良好。"  
),
(
    "スーパーホテル山形・さくらんぼ東根駅前 天然温泉桜桃の湯",  
    "4,000円~/人",  
    "山形県東根市さくらんぼ駅前2丁目17番21号",  
    "JRさくらんぼ東根駅より徒歩3分!新幹線や電車、飛行機でも!無料先着駐車場完備!"  
),
(
    "ダイヤモンドホテル",  
    "7,500円~/人",  
    "東京都千代田区麹町1-10-3",  
    "半蔵門線半蔵門駅(3a・4番出口)徒歩1分。有楽町線麹町駅徒歩5分。"  
),
(
    "ホテルルートイン東京池袋",  
    "5,500円~/人",  
    "東京都豊島区東池袋3-5-5",  
    "池袋駅東口より徒歩6分/首都高5号線東池袋ICより約2分"  
),
(
    "ガストホフ あみ",  
    "10,300円~/人",  
    "栃木県日光市久次良町100",  
    "日光宇都宮道路日光ICより約10分。★国道120号線から必ずあみ看板を入る。★間違えるナビが多いです。"  
),
(
    "アパホテル<秋葉原駅電気街口>",  
    "5,350円~/人",  
    "東京都千代田区外神田3丁目11-4",  
    "JR山手線・京浜東北線・総武線「秋葉原駅」電気街口から徒歩3分!東京メトロ銀座線「末広町駅」から徒歩2分"  
),
(
    "ホテルマイステイズ神田",  
    "4,200円~/人",  
    "東京都千代田区岩本町1-2-2",  
    "JR・地下鉄神田駅 南口徒歩6分/JR新日本橋駅 8番出口徒歩3分/地下鉄小伝馬町駅 4番出口徒歩6分"  
),
(
    "カリオ笹塚テラス",  
    "9,660円~/人",  
    "東京都渋谷区笹塚1-56-7",  
    "新宿駅から京王線で5分、笹塚駅から北へ徒歩1分"  
),
(
    "Minn蒲田",  
    "5,460円~/人",  
    "東京都大田区西蒲田5-18-18",  
    "JR蒲田駅西口より徒歩で約5分、京急蒲田駅 徒歩で約16分"  
),
(
    "ホテルサンターガス上野店",  
    "1円~/人",  
    "東京都台東区東上野2-19-3",  
    "JR上野駅広小路口より徒歩4分、東京メトロ上野駅3番出口より徒歩3分"  
),
(
    "上野東金屋ホテル",  
    "4,950円~/人",  
    "東京都台東区東上野3―17―5",  
    "JR上野駅徒歩2分、地下鉄上野駅2番出口より徒歩30秒。京成上野駅徒歩7分。"  
),
(
    "コートヤード・バイ・マリオット 東京ステーション",  
    "36,700円~/人",  
    "東京都中央区京橋2-1-3",  
    "東京駅八重洲南口から徒歩4分 東京メトロ京橋駅7番出口より徒歩1分 都営浅草線 宝町駅A5出口より徒歩4分"  
),
(
    "相鉄フレッサイン日本橋茅場町",  
    "4,800円~/人",  
    "東京都中央区日本橋茅場町1-3-10",  
    "東西線・日比谷線茅場町駅7番出口(エレベーターなし)徒歩1分・4a出口(エレベーターあり)徒歩2分"  
),
(
    "東京ビュック",  
    "6,050円~/人",  
    "東京都中央区勝どき2―8-12",  
    "都営大江戸線勝どき駅(A2a・A2b出口)より徒歩1分。都バス勝どき駅前バス停正面。"  
),
(
    "ホテル庵築地",  
    "6,975円~/人",  
    "東京都中央区築地3-10-4",  
    "築地駅3番口から徒歩1分、築地駅4番口から徒歩3分と交通に大変便利。"  
),
(
    "SHIBUYA HOTEL EN(渋谷ホテル エン)",  
    "10,430円~/人",  
    "東京都渋谷区円山町1-1",  
    "JR渋谷駅(ハチ公口)より徒歩7分、地下鉄半蔵門線3A出口より徒歩6分。京王電鉄井の頭線神泉駅徒歩5分。"  
),
(
    "ホテルトムス",  
    "3,895円~/人",  
    "東京都大田区西蒲田7-21-5",  
    "JR蒲田駅西口より徒歩約7分、東急池上線蓮沼駅より徒歩約3分"  
),
(
    "海辺のコンドミ ごんべえ",  
    "6,050円~/人",  
    "千葉県南房総市久枝769",  
    "車:館山自動車道 鋸南富山ICを降りて信号2つ目を右折→二部交差点を左折→JASSを過ぎて右側に看板あり"  
),
(
    "HANABI HOTEL",  
    "3,800円~/人",  
    "東京都新宿区百人町2-8-5",  
    "新大久保駅より徒歩2分"  
),
(
    "ますがたや旅館",  
    "5,000円~/人",  
    "千葉県香取郡多古町多古970",  
    "成田空港より路線バス約15分 【多古町役場前】徒歩5分"  
),
(
    "The.Beach.Inn.SHIRAKO",  
    "19,000円~/人",  
    "千葉県長生郡白子町古所3283ー4",  
    "九十九里有料道路"  
),
(
    "ニューパークホテル",  
    "4,500円~/人",  
    "東京都台東区東上野2-19-5",  
    "JR上野駅、広小路口より徒歩4分。東京メトロ地下鉄3番出口より徒歩2分。京成上野駅より徒歩7分。"  
),
(
    "ホテルシロー",  
    "5,500円~/人",  
    "千葉県船橋市本町2-5-14",  
    "JR船橋駅下車南口より徒歩7分。 京成船橋駅下車徒歩5分。 京葉道路船橋ICより3分。"  
),
(
    "ホテルマイステイズ舞浜",  
    "10,400円~/人",  
    "千葉県浦安市鉄鋼通り3-5-1",  
    "JR「舞浜駅」より徒歩12分。車...首都高湾岸線「葛西ランプ」または「浦安ランプ」下車5分。"  
),
(
    "ナイスインホテル市川東京ベイ~舞浜まで電車6分~",  
    "1円~/室",  
    "千葉県市川市塩浜3-10-31",  
    "舞浜駅(東京ディズニーリゾート最寄駅)まで電車で2駅6分の好立地♪当ホテルも駅ロータリー目の前です!♪"  
),
(
    "成田山門前 旅館 若松本店",  
    "11,550円~/人",  
    "千葉県成田市本町355",  
    "JRまたは京成成田駅より徒歩15分。またはタクシー5分。成田空港よりお車約15分。"  
),
(
    "オリンピックイン神田",  
    "4,300円~/人",  
    "東京都千代田区神田富山町30-1",  
    "JR神田駅東口またはメトロ銀座線神田駅3番出口より徒歩4分。都営地下鉄岩本町駅A1出口より徒歩4分。"  
),
(
    "パールホテル茅場町",  
    "8,250円~/人",  
    "東京都中央区新川1-2-5",  
    "茅場町駅4b出口より徒歩3分、東京駅からタクシーで5分"  
),
(
    "ホテル カアナパリ",  
    "5,720円~/人",  
    "千葉県長生郡白子町中里4519",  
    "都心から90分白子温泉リゾート"  
),
(
    "オークホテル江戸",  
    "6,250円~/人",  
    "東京都江東区森下2-17-7",  
    "東京森下駅A6番出口を出て右折し直進、最初の交差点をまた右折直進しますとホテルが右側にございます。"  
),
(
    "瑞江第一ホテル",  
    "4,550円~/人",  
    "東京都江戸川区瑞江2-6-16",  
    "電車もいいけど車も便利!首都高一之江・京葉篠崎各ICから10分 外環接続で関越・東北・常磐各方面も楽々!"  
),
(
    "変なホテル東京銀座",  
    "7,690円~/人",  
    "東京都中央区築地2―2―1",  
    "東京メトロ新富町駅 徒歩2分,築地駅 徒歩5分,銀座駅 徒歩16分,東京ディズニーリゾート(R)電車 約20分♪"  
),
(
    "相鉄フレッサイン東京神田",  
    "7,245円~/人",  
    "東京都千代田区神田紺屋町8-1",  
    "JR神田駅南口から徒歩4分!東京駅と秋葉原駅は両隣りでアクセス便利◆ホテルはセブンイレブンのすぐ手前!"  
),
(
    "WOODLAND BOTHY",  
    "25,300円~/人",  
    "東京都あきる野市養沢684番地",  
    "JR 五日市線「武蔵五日市駅」→上養沢行きバス「神谷バス停」30分→登山20分。八王子IC→当駐車場30分。"  
),
(
    "恵比寿ホリックホテル",  
    "14,240円~/人",  
    "東京都渋谷区恵比寿南1丁目16-8",  
    "JR山手線/恵比寿駅西口より徒歩5分。地下鉄日比谷線/恵比寿駅1番出口より徒歩5分。"  
),
(
    "ホテル ドエル",  
    "5,250円~/人",  
    "千葉県八千代市勝田台北1-11-11",  
    "京成線、東葉高速鉄道勝田台駅A3出口より徒歩3分。"  
),
(
    "チサンホテル浜松町",  
    "5,625円~/人",  
    "東京都港区芝浦1丁目3番10号",  
    "JR「浜松町駅」南口より徒歩8分(連絡通路を浜松町ビルディング(旧東芝ビル)方面へモノレール線路沿い。"  
),
(
    "ビジネスホテルサンパレス",  
    "5,650円~/人",  
    "東京都荒川区南千住2-32-3",  
    "JR南千住駅南口から歩道橋経由で徒歩約3分/都電荒川線 三ノ輪橋駅まで徒歩圏内"  
),
(
    "ホテルリズベリオ赤坂",  
    "13,950円~/人",  
    "東京都港区赤坂3-18-1",  
    "東京メトロ丸ノ内線・銀座線赤坂見附駅10番出口より徒歩2分/ディズニーランドまで40分/東京ドームまで20分"  
),
(
    "アーバンホテル三幸",  
    "3,400円~/人",  
    "千葉県千葉市中央区今井2-11-8",  
    "JR京葉線始発駅・内房線・外房線・特急停車駅の蘇我駅下車、西口徒歩2分"  
),
(
    "THE skM TOKYO HOTEL&DINING",  
    "67,850円~/室",  
    "東京都墨田区江東橋4-25-5",  
    "JR中央線/総武線/総武快速線/半蔵門線錦糸町駅より 徒歩3分"  
),
(
    "渋谷クレストンホテル(HMIホテルグループ)",  
    "12,700円~/人",  
    "東京都渋谷区神山町10-8",  
    "渋谷駅ハチ公口より文化村通りをNHK方面へ直進、神山町東の信号を右折し、30m先左手。"  
),
(
    "IKIDANE Residential Hotel 墨田京島",  
    "4,888円~/人",  
    "東京都墨田区京島3-32-10",  
    "押上駅より徒歩にて約10分、羽田空港から直行で約40分です。"  
),
(
    "松山荘",  
    "4,400円~/人",  
    "千葉県南房総市久枝512‐1",  
    "鋸南富山ICより車で8分・JR岩井駅より徒歩10分"  
),
(
    "ホテルリブマックス葛西駅前",  
    "3,825円~/人",  
    "東京都江戸川区東葛西6-1-3",  
    "葛西駅 中央口目の前の大きな道を右へ3分。業務スーパーの向かいにホテルがございます。"  
),
(
    "ホテルシュランザMAKUHARI BAY",  
    "8,225円~/人",  
    "千葉県千葉市美浜区ひび野2-7-4",  
    "海浜幕張駅南口徒歩2分|舞浜駅まで約20分|幕張メッセまで徒歩約10分"  
),
(
    "リッチモンドホテルプレミア東京スコーレ(旧:プレミア東京押上)",  
    "11,400円~/人",  
    "東京都墨田区押上1-10-3",  
    "押上駅(半蔵門線・都営浅草線・京成押上線・東武線)から徒歩1分。 とうきょうスカイツリー駅から徒歩3分"  
),
(
    "ホテルマイステイズ五反田駅前",  
    "5,600円~/人",  
    "東京都品川区西五反田2-6-8",  
    "五反田駅西口から徒歩30秒。JR山手線・都営浅草線・東急池上線が利用可で渋谷まで8分、品川まで5分!"  
),
(
    "ダイワロイネットホテル広島",  
    "4,250円~/人",  
    "広島県広島市中区国泰寺町1-3-20",  
    "JR広島駅より路面電車1番・広島港行き(約20分)「中電前」下車、徒歩約1分。"  
),
(
    "KKRホテル東京(国家公務員共済組合連合会東京共済会館)",  
    "14,300円~/人",  
    "東京都千代田区大手町1-4-1",  
    "地下鉄東西線竹橋駅3b出口直結。東京駅丸の内北口タクシーワンメーター。大手町駅神保町駅より徒歩5分"  
),
(
    "渋谷エクセルホテル東急",  
    "15,740円~/人",  
    "東京都渋谷区道玄坂1-12-2",  
    "【電車】JR/銀座・半蔵門線/東横・田園都市線、井の頭線渋谷駅直結。【車】首都高渋谷ICから車3分。"  
),
(
    "ホテル ルミエール葛西",  
    "5,050円~/人",  
    "東京都江戸川区中葛西5-41-20",  
    "東京メトロ東西線葛西駅下車徒歩2分"  
),
(
    "ホテルサンルート浅草",  
    "4,900円~/人",  
    "東京都台東区雷門1-8-5",  
    "東京メトロ銀座線田原町駅徒歩1分。都営浅草線浅草駅徒歩7分。首都高速6号線駒形IC3分。"  
),
(
    "ペンションOwl(アウル)姫木",  
    "7,150円~/人",  
    "長野県小県郡長和町大門姫木平3518-2638",  
    "中央道から諏訪IC下車にてR152にて40分関越から佐久IC下車にてR142-R152経由50分"  
),
(
    "ロワジールホテル品川シーサイド",  
    "4,230円~/人",  
    "東京都品川区東品川4-12-8",  
    "りんかい線「品川シーサイド駅」下車、C出口より徒歩2分、京浜急行線「青物横丁駅」より徒歩7分"  
),
(
    "東武ホテルレバント東京 東京スカイツリー(R)オフィシャルホテル",  
    "10,290円~/人",  
    "東京都墨田区錦糸1-2-2",  
    "JR錦糸町駅北口下車徒歩3分、地下鉄半蔵門線錦糸町駅下車徒歩4分。首都高速7号線錦糸町出口より5分。"  
),
(
    "桜スカイホテル",  
    "6,800円~/人",  
    "東京都墨田区江東橋4-9-3",  
    "JR総武快速・総武各停、東京メトロ半蔵門線 錦糸町駅 より徒歩にて約5分"  
),
(
    "箱根水明荘",  
    "22,550円~/人",  
    "神奈川県足柄下郡箱根町湯本702",  
    "小田急、箱根登山線箱根湯本駅より徒歩2分"  
),
(
    "白子ニューシーサイドホテル",  
    "5,500円~/人",  
    "千葉県長生郡白子町中里4482",  
    "JR外房線茂原駅下車白子車庫行きバス20分、中里海岸下車、徒歩2分。"  
),
(
    "JR東日本ホテルメッツ 津田沼",  
    "3,960円~/人",  
    "千葉県習志野市津田沼1-1-1",  
    "JR津田沼駅南口徒歩1分 京成津田沼駅から徒歩15分・バス5分"  
),
(
    "ホテルマイステイズ羽田",  
    "5,100円~/人",  
    "東京都大田区羽田5-1-13",  
    "京急空港線「穴守稲荷」駅より徒歩4分。東京モノレール「天空橋」駅からは徒歩8分。"  
),
(
    "相鉄フレッサイン東京田町",  
    "4,080円~/人",  
    "東京都港区芝浦3-14-21",  
    "JR田町駅「芝浦口」より徒歩4分。都営三田線三田駅下車徒歩7分。ゆりかもめ芝浦ふ頭駅下車15分"  
),
(
    "マイステイズ新浦安",  
    "5,000円~/人",  
    "千葉県浦安市明海2-1-4",  
    "JR京葉線・武蔵野線「新浦安」駅下車 南口(バスターミナル側)より徒歩約15分 タクシー約5分"  
),
(
    "HOTEL TABARD TOKYO",  
    "7,600円~/人",  
    "東京都墨田区江東橋4-14-5",  
    "JR総武線・総武快速線・東京メトロ半蔵門線『錦糸町駅』徒歩5分"  
),
(
    "目黒ホリックホテル",  
    "7,600円~/人",  
    "東京都目黒区目黒1-3-14",  
    "JR目黒駅西口より徒歩3分、西口を出て右方向へ、一つ目の信号を左へ、権之助坂を下ってすぐ。"  
),
(
    "プルマン東京田町",  
    "30,400円~/人",  
    "東京都港区芝浦3-1-21",  
    "JR田町駅東口より徒歩1分 msb田町(ムスブ)敷地内、地下鉄三田駅からA4出口より徒歩3分"  
),
(
    "アーヴェストホテル蒲田",  
    "4,750円~/人",  
    "東京都大田区蒲田5-12-12",  
    "JR蒲田駅東口より徒歩約1分 /京急蒲田駅西口より徒歩約13分"  
),
(
    "ファミーINN幕張",  
    "4,050円~/人",  
    "千葉県千葉市花見川区幕張本郷1-33-3",  
    "JR・京成幕張本郷駅下車徒歩7分 京葉道路幕張インターより約3分"  
),
(
    "スカイハートホテル京成小岩駅前",  
    "5,600円~/人",  
    "東京都江戸川区北小岩6-11-4",  
    "京成線京成小岩駅北口より徒歩1分。JR小岩からは京成バスで10分。"  
),
(
    "スマイルホテル日本橋三越前",  
    "4,900円~/人",  
    "東京都中央区日本橋本町1-4-14",  
    "三越前駅A1又はA4出口より徒歩2分。東京駅八重洲口徒歩15分"  
),
(
    "ホテルリブマックス日本橋箱崎",  
    "4,908円~/人",  
    "東京都中央区日本橋箱崎町29-5",  
    "水天宮前駅2番出口から徒歩2分/人形町駅・茅場町駅から約10分/T-CATから羽田まで約30分"  
),
(
    "ホテルルートイン品川大井町",  
    "8,700円~/室",  
    "東京都品川区大井1-35-2",  
    "JR京浜東北線(中央口)・東急大井町線(A2出口)・りんかい線、大井町駅より徒歩5分"  
),
(
    "チサンホテル蒲田",  
    "4,450円~/人",  
    "東京都大田区西蒲田8-20-11",  
    "羽田空港から最短30分♪品川駅には最短10分♪コインランドリー等の設備も充実で観光の拠点にもピッタリ♪"  
),
(
    "レッドルーフ イン蒲田/羽田 東京",  
    "4,095円~/人",  
    "東京都大田区西蒲田7-24-7",  
    "JR蒲田駅西口から徒歩5分♪横浜、品川までは乗り換えなし!羽田空港までも駅からのシャトルバスで約30分♪"  
),
(
    "BAN HOTEL 築地銀座",  
    "4,400円~/人",  
    "東京都中央区築地3-17-10",  
    "日比谷線築地駅 1番、2番出口 徒歩3分 都営大江戸線 築地市場駅 A1出口 徒歩5分"  
),
(
    "ReLA東松戸",  
    "5,444円~/人",  
    "千葉県松戸市東松戸1丁目3番2",  
    "東松戸駅から徒歩3分"  
),
(
    "高濃度炭酸泉 ハナミズキの湯 スーパーホテル新横浜",  
    "4,650円~/人",  
    "神奈川県横浜市港北区新横浜2-6-20",  
    "駅近好立地♪♪東海道新幹線新横浜駅北口から3分、地下鉄新横浜駅8番出口から3分、第三京浜港北IC車で約5分"  
),
(
    "ハートンホテル東品川(品川シーサイド)",  
    "4,640円~/人",  
    "東京都品川区東品川4-13-27",  
    "りんかい線「品川シーサイド駅」A出口徒歩1分。京浜急行線「青物横丁駅」徒歩10分"  
),
(
    "L stay&grow晴海",  
    "4,270円~/人",  
    "東京都中央区晴海3-8-1",  
    "都営地下鉄大江戸線 勝どき駅A5出口より徒歩3分、ゆりかもめ新豊洲駅より徒歩15分。"  
),
(
    "京成ホテルミラマーレ",  
    "4,600円~/人",  
    "千葉県千葉市中央区本千葉町15-1",  
    "京成千葉中央駅直結、JR千葉駅から徒歩約8分。有料駐車場有。"  
),
(
    "三井ガーデンホテル豊洲プレミア",  
    "28,100円~/人",  
    "東京都江東区豊洲2-2-1",  
    "東京メトロ有楽町線豊洲駅(2番出口)直結"  
),
(
    "Jホテル東京ジオ",  
    "4,950円~/人",  
    "東京都中央区日本橋本石町3-1-6",  
    "東京駅、神田駅、大手町、三越前駅、徒歩5分~7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速3駅14分可"  
),
(
    "ホテルルートイン千葉浜野-東京湾岸道-",  
    "4,750円~/人",  
    "千葉県千葉市中央区浜野町1025-110",  
    "京葉道路・館山自動車道 蘇我ICより3.5km・車で約9分◆JR内房線 浜野駅より690m 徒歩約8分・車で約4分"  
),
(
    "東京ベイ有明ワシントンホテル",  
    "5,581円~/人",  
    "東京都江東区有明3-7-11",  
    "りんかい線「国際展示場駅」下車徒歩3分。新交通ゆりかもめ「東京ビッグサイト駅」「有明駅」下車徒歩3分。"  
),
(
    "JR東日本ホテルメッツ 船橋",  
    "4,300円~/人",  
    "千葉県船橋市本町7丁目1-1",  
    "JR船橋駅から徒歩1分 シャポー船橋南館6階"  
),
(
    "ホテルトラスティ東京ベイサイド",  
    "5,500円~/人",  
    "東京都江東区有明3-1-15",  
    "東京ビッグサイト駅徒歩5分/国際展示場駅から徒歩10分の好立地!駐車場あり★ビジネスにも観光にも便利★"  
),
(
    "ダイワロイネットホテル岡山駅前",  
    "5,500円~/人",  
    "岡山県岡山市北区駅前町1-1-1",  
    "JR岡山駅後楽園口(東口)より徒歩1分。"  
),
(
    "ホテルマークワンCNT",  
    "5,500円~/人",  
    "千葉県印西市中央南1-10",  
    "特急停車駅【北総線】千葉ニュータウン中央駅より徒歩3分、成田空港より4駅、乗り換えなし"  
),
(
    "スーパーホテル東西線・市川・妙典駅前",  
    "4,350円~/人",  
    "千葉県市川市妙典4-4-27妙典センタービル",  
    "電車/東西線妙典駅下車 徒歩2分 車/市川ICより約7分"  
),
(
    "リッチモンドホテル成田",  
    "4,375円~/人",  
    "千葉県成田市花崎町970",  
    "京成線京成成田駅東口より徒歩5分、JR成田駅東口より徒歩10分、成田空港駅より2駅"  
),
(
    "京急EXホテル高輪(旧京急EXイン高輪)",  
    "8,550円~/人",  
    "東京都港区高輪4-10-8",  
    "品川駅高輪口より徒歩3分"  
),
(
    "相鉄フレッサイン東京錦糸町",  
    "4,860円~/人",  
    "東京都墨田区江東橋2-18-7",  
    "JR・東京メトロ錦糸町駅から徒歩3分"  
),
(
    "WeskiiHotel(ウェスキーホテル)",  
    "13,300円~/人",  
    "石川県金沢市広岡1丁目14-6",  
    "JR『金沢』駅 西口より徒歩約3分/無料駐車場を完備"  
),
(
    "京急EXイン浜松町・大門駅前",  
    "5,450円~/人",  
    "東京都港区芝大門1-15-4",  
    "浜松町駅北口徒歩5分/大門駅A5出口徒歩1分/御成門駅徒歩7分/新橋駅徒歩13分/田町駅徒歩22分"  
),
(
    "ファーイーストビレッジホテル東京有明",  
    "5,340円~/人",  
    "東京都江東区有明1丁目2番43号",  
    "ゆりかもめ有明テニスの森駅より徒歩3分、首都高湾岸線有明出口より3分。"  
),
(
    "お茶の水イン〈全館禁煙〉",  
    "4,750円~/人",  
    "東京都文京区湯島1-3-7",  
    "JRお茶の水駅、地下鉄丸ノ内線御茶ノ水駅、千代田線新御茶ノ水駅、それぞれの駅より徒歩3分。"  
),
(
    "サイプレスイン東京",  
    "3,700円~/人",  
    "東京都葛飾区西新小岩1-2-5",  
    "新小岩駅徒歩1分  東京駅→新小岩駅総武本線13分 秋葉原駅→新小岩駅総武線13分"  
),
(
    "市川グランドホテル(BBHホテルグループ)",  
    "3,350円~/人",  
    "千葉県市川市市川1-3-18  SRビル市川(1階千葉興業銀行)6階フロント",  
    "JR総武線市川駅北口より徒歩3分 入口は千葉街道線沿い1F千葉興銀の6Fフロント"  
),
(
    "京急EXイン秋葉原",  
    "5,500円~/人",  
    "東京都台東区秋葉原2-1",  
    "JR秋葉原駅(中央改札口)・末広町駅 どちらの駅からも徒歩約5分"  
),
(
    "ホテルファミーINN・錦糸町",  
    "4,750円~/人",  
    "東京都墨田区江東橋2-17-4",  
    "JR総武線錦糸町駅南口・東京メトロ半蔵門線錦糸町駅1番出口より徒歩1分"  
),
(
    "都市センターホテル(リーガロイヤルホテルグループ)",  
    "7,245円~/人",  
    "東京都千代田区平河町2―4―1 日本都市センター会館",  
    "東京メトロ 永田町駅4・5・9b出口徒歩4分。麹町駅1出口徒歩4分。赤坂見附駅D出口徒歩8分。"  
),
(
    "ザ キューブホテル千葉",  
    "5,266円~/人",  
    "千葉県千葉市中央区問屋町1-45 千葉ポートスクエア",  
    "千葉都市モノレール 市役所前駅から徒歩で約7分。 JR京葉線 千葉みなと駅から徒歩で約15分。駐車場完備!"  
),
(
    "ホテル市松",  
    "5,000円~/人",  
    "千葉県船橋市宮本3―1―3",  
    "京成大神宮下駅より徒歩約3分、JR船橋駅・京葉線南船橋駅より徒歩約15分、船橋IC、花輪ICから約5分"  
),
(
    "アルモントイン東京日本橋",  
    "5,700円~/人",  
    "東京都中央区日本橋小網町4-6",  
    "茅場町駅4a出口、水天宮前駅6番出口より徒歩5分。東京駅より車で約10分。T-CATより徒歩約10分。"  
),
(
    "上野ファーストシティホテル",  
    "5,130円~/人",  
    "東京都台東区上野1-14ー8",  
    "JR御徒町駅徒歩5分、地下鉄上野御徒町駅、上野広小路駅、仲御徒町駅各徒歩4、湯島駅徒歩1分。"  
),
(
    "相鉄フレッサイン 大門駅前(2022年1月26日新規オープン)",  
    "5,312円~/人",  
    "東京都港区芝大門2-6-2",  
    "都営地下鉄浅草線・大江戸線 大門駅出口A3より徒歩2分・JR山手線・京浜東北線 浜松町駅南口より徒歩5分"  
),
(
    "ホテル グレイスリー田町",  
    "7,100円~/人",  
    "東京都港区芝浦3-8-1",  
    "JR田町駅「芝浦口」より徒歩4分【電車で羽田空港まで25分・新宿駅まで22分の好アクセス!】"  
),
(
    "アパホテル<TKP東京西葛西>",  
    "4,200円~/人",  
    "東京都江戸川区西葛西6丁目15-24",  
    "東京地下鉄(東京メトロ)東西線「西葛西駅」南口から徒歩1分"  
),
(
    "ホテルミッドイン蔵前駅前(2021年7月15日オープン)",  
    "6,000円~/人",  
    "東京都台東区蔵前3-1-1",  
    "都営浅草線蔵前駅A3口直結/都営大江戸線蔵前駅A6口徒歩5分/JR浅草橋駅徒歩9分/浅草駅より徒歩15分"  
),
(
    "ホテルアーバン",  
    "4,950円~/人",  
    "東京都豊島区西池袋3-26-6",  
    "池袋駅西口から徒歩2分 地下道1b出口(有楽町線寄り)から徒歩1分※22時以降の最寄出口は『C7』"  
),
(
    "相鉄フレッサイン浜松町大門",  
    "4,185円~/人",  
    "東京都港区芝大門1-2-7",  
    "JR浜松町駅北口より徒歩8分 都営浅草・大江戸線大門駅A5出口より徒歩4分 都営三田線御成門駅より徒歩4分"  
),
(
    "ホテルイルフィオーレ葛西",  
    "4,400円~/人",  
    "東京都江戸川区東葛西6-5-8",  
    "東京メトロ東西線・葛西駅 中央口右手出て徒歩2分 主要観光スポットへのバスのアクセスもとても便利!"  
),
(
    "ホテルミッドイン目黒駅前",  
    "7,800円~/人",  
    "東京都目黒区下目黒1-2-19",  
    "JR・地下鉄・東急線の目黒駅より徒歩3分!!"  
),
(
    "ORDA HOTEL",  
    "3,870円~/人",  
    "千葉県千葉市中央区中央3-6-1",  
    "■京成「千葉中央駅」東口徒歩約7分■JR「千葉駅」東口徒歩約10分■モノレール「葭川公園駅」東側徒歩約3分"  
),
(
    "ホテルマイステイズ亀戸",  
    "2,800円~/人",  
    "東京都江東区亀戸6-32-1",  
    "JR「亀戸駅」東口より徒歩4分/都営新宿線「西大島駅」より徒歩10分"  
),
(
    "ザ・スクエアホテル銀座",  
    "11,800円~/人",  
    "東京都中央区銀座2-11-6",  
    "■銀座一丁目駅11番出口(7:00-22:00)より徒歩1分■東銀座駅A2出口より徒歩5分■有楽町駅京橋口より徒歩8分"  
),
(
    "サンホテル船橋",  
    "5,500円~/人",  
    "千葉県船橋市本町6-21-18",  
    "JR船橋駅北口から徒歩5分!京葉道船橋IC5分!"  
),
(
    "WE HOME HOTEL&KITCHEN 市川・船橋",  
    "3,500円~/人",  
    "千葉県市川市鬼高2-10-2",  
    "◆JR下総中山駅(南口)徒歩3分◆舞浜(東京ディズニーR)・海浜幕張(幕張メッセ)まで共に4駅の好立地◆"  
),
(
    "くれたけインプレミアム浜松町",  
    "6,650円~/人",  
    "東京都港区芝1-8-18",  
    "JR山手線・京浜東北線 浜松町駅 南口改札より徒歩8分"  
),
(
    "フレックステイイン川崎貝塚",  
    "3,850円~/人",  
    "神奈川県川崎市川崎区貝塚1-13-2",  
    "JR「川崎駅」東口より徒歩10分。京浜急行線「京急川崎駅」本口より徒歩13分。"  
),
(
    "ホテルミッドイン川崎駅前",  
    "6,200円~/人",  
    "神奈川県川崎市川崎区砂子2-7-7",  
    "JR川崎駅東口・京浜急行京急川崎駅中央改札より徒歩約6分です。"  
),
(
    "ホテルアベスト目黒",  
    "5,550円~/人",  
    "東京都品川区上大崎2-26-5",  
    "山手線/メトロ南北線/都営三田線/東急目黒線/目黒駅より徒歩1分。品川/渋谷/新宿/都心へのアクセス良好。"  
),
(
    "デイナイスホテル東京",  
    "4,400円~/人",  
    "東京都江東区木場2-1-1",  
    "東京メトロ東西線木場駅4a番出口より徒歩約3分、門前仲町駅2番出口より徒歩約5分。"  
),
(
    "東京イン",  
    "5,000円~/人",  
    "東京都大田区北馬込2-31-6",  
    "都営地下鉄浅草線・馬込駅A3出口隣接♪"  
),
(
    "ホテル イルフィオーレ葛西 ANNEX",  
    "5,100円~/人",  
    "東京都江戸川区東葛西6-1-7",  
    "東京メトロ東西線・葛西駅中央改札右手出て徒歩5分。主要観光スポットへのアクセスも便利!"  
),
(
    "ビジネスホテル レンド",  
    "4,700円~/人",  
    "埼玉県川口市東川口2-1-22",  
    "JR武蔵野線、埼玉高速鉄道 東川口駅下車徒歩3分。お車→浦和ICから10分。"  
),
(
    "ホテルリブマックス茅場町",  
    "4,888円~/人",  
    "東京都中央区日本橋茅場町3-7-3",  
    "東京メトロ東西線/日比谷線・茅場町駅2番出口より徒歩1分、八重洲南口のバス10番乗場より亀嶋橋下車約3分"  
),
(
    "スーパーホテルPremier東京駅八重洲中央口 高濃度炭酸泉",  
    "11,850円~/人",  
    "東京都中央区八重洲2-2-7",  
    "東京駅八重洲中央口を出て横断歩道を渡り直進し、一つ目の信号を右に曲がるとすぐにございます。"  
),
(
    "秀花園 湯の花膳",  
    "22,500円~/人",  
    "静岡県熱海市和田浜南町7-13",  
    "【じゃらんOF THE YEAR受賞】熱海駅より送迎バス有【要予約】熱海駅より路線バス、タクシー7分"  
),
(
    "品川プリンスホテル",  
    "8,980円~/人",  
    "東京都港区高輪4-10-30",  
    "新幹線・JR線・京急線 品川駅(高輪口)より徒歩約2分。京急線 羽田空港国内線ターミナル駅より最速14分!"  
),
(
    "相鉄グランドフレッサ 東京ベイ有明(旧ホテルサンルート有明)",  
    "5,935円~/人",  
    "東京都江東区有明3-6-6",  
    "りんかい線国際展示場駅、ゆりかもめ東京ビッグサイト駅から徒歩3分。首都高湾岸線有明ランプより3分。"  
),
(
    "ニューオータニイン東京",  
    "5,940円~/人",  
    "東京都品川区大崎1―6―2",  
    "JR山手線・りんかい線 大崎駅北改札口直結。お台場へ11分、舞浜駅へ24分、横浜ベイエリア30分!"  
),
(
    "熱海温泉 湯宿 みかんの木",  
    "16,900円~/人",  
    "静岡県熱海市東海岸町1-11",  
    "JR熱海駅改札より徒歩約12分。坂の途中にあるため御御足が悪い方はタクシーがオススメです。"  
),
(
    "ロイヤルパークホテル",  
    "11,200円~/人",  
    "東京都中央区日本橋蛎殻町2-1-1",  
    "東京メトロ半蔵門線「水天宮前駅」直結/日比谷線・都営浅草線「人形町駅」徒歩圏内"  
),
(
    "HOTEL HOUSEN ホテル朋泉 草加(埼玉県)",  
    "4,400円~/人",  
    "埼玉県草加市住吉1-2-27",  
    "東京メトロ日比谷線・半蔵門線直通・東武スカイツリーライン(旧伊勢崎線)草加駅東口より徒歩3分。"  
),
(
    "京急EXイン東京・日本橋(2020年4月7日新規オープン)",  
    "7,240円~/人",  
    "東京都中央区日本橋茅場町2-4-8",  
    "茅場町駅2番出口徒歩約2分/八丁堀駅A5徒歩約4分/日本橋駅D1出口徒歩約6分/東京駅八重洲中央口徒歩約14分"  
),
(
    "和空 成田山門前",  
    "16,905円~/人",  
    "千葉県成田市仲町379-1",  
    "「京成電鉄 京成成田駅」「JR線 成田駅」より徒歩10分、成田山新勝寺まで徒歩約2分"  
),
(
    "ホテルサンルート千葉",  
    "4,700円~/人",  
    "千葉県千葉市中央区新千葉1丁目4番地1号ウェストリオ",  
    "JR千葉駅直結-西口改札より徒歩1分。中央改札より徒歩3分。千葉都市モノレール千葉駅より徒歩5分。"  
),
(
    "京急EXイン京急川崎駅前",  
    "4,700円~/人",  
    "神奈川県川崎市川崎区砂子1-3-1駅ビル(Wing kitchen)3F",  
    "京急川崎駅直結 徒歩1分(駅ビル Wing kitchen 3F)JR川崎駅「中央改札東口」より徒歩5分。"  
),
(
    "ハイアット リージェンシー 東京ベイ",  
    "9,800円~/人",  
    "千葉県浦安市明海5-8-23",  
    "ホテル無料シャトルバスあり ■JR新浦安駅より約10分 ■東京ディズニーリゾート(R)より約25分"  
),
(
    "日和(ひより)ホテル舞浜",  
    "8,750円~/人",  
    "千葉県浦安市東野2-25-8",  
    "JR舞浜駅南口「C3」乗り場より無料シャトルバスで約10分。首都高速湾岸線「浦安」出口すぐ。"  
),
(
    "京急EXイン京急蒲田駅前",  
    "4,640円~/人",  
    "東京都大田区蒲田4丁目45-3",  
    "羽田空港国際線ターミナル駅から京急空港線で最短8分 京急蒲田駅から徒歩3分"  
),
(
    "京成リッチモンドホテル東京錦糸町",  
    "6,750円~/人",  
    "東京都墨田区江東橋3-2-7",  
    "各線【錦糸町駅】JR総武線南口・東京メトロ半蔵門線1番出口より徒歩5分"  
),
(
    "スーパーホテル東京・日本橋三越前",  
    "5,400円~/人",  
    "東京都中央区日本橋小舟町14-13",  
    "人形町駅A5出口・小伝馬町駅3番出口から徒歩約5分/三越前駅A5出口・新日本橋駅5番出口から徒歩約6分"  
),
(
    "東京ビジネスホテル",  
    "6,380円~/人",  
    "東京都新宿区新宿6―3―2",  
    "新宿駅より徒歩15分、タクシー5分。新宿御苑前駅、新宿三丁目駅より徒歩7分。 バスタ新宿より徒歩15分"  
),
(
    "川崎日航ホテル",  
    "5,400円~/人",  
    "神奈川県川崎市川崎区日進町1",  
    "JR川崎駅に隣接し、羽田空港からわずか15分。アゼリア地下街直結なので雨天時でも安心。"  
),
(
    "スーパーホテル秋葉原・末広町 高濃度炭酸泉 藍染の湯",  
    "5,510円~/人",  
    "東京都台東区上野3-2-5",  
    "東京メトロ銀座線「末広町」駅徒歩2分、山手線「御徒町」駅南口徒歩5分、「秋葉原」駅電気街口徒歩7分"  
),
(
    "トーセイホテル&セミナー幕張",  
    "3,500円~/人",  
    "千葉県習志野市茜浜2-3-2",  
    "京葉線新習志野駅より徒歩2分!! ビジネスや観光の拠点に最適です♪ 35台の平面駐車場完備♪"  
),
(
    "hotel it.osaka shinmachi",  
    "6,600円~/人",  
    "大阪府大阪市西区新町4丁目6-20",  
    "大阪メトロ千日前線・長堀鶴見緑地線西長堀駅下車4番A出口より徒歩2分、中央線阿波座駅5番出口徒歩約8分"  
),
(
    "キャメルホテル&グランピングリゾート",  
    "7,500円~/人",  
    "千葉県夷隅郡御宿町上布施3360",  
    "お車:圏央道市原鶴舞IC下車35分 電車:外房線御宿駅よりタクシー13分※要事前予約"  
),
(
    "カジュアルフレンチを愉しむ 眺望山荘",  
    "10,700円~/人",  
    "神奈川県足柄下郡湯河原町宮下698番地の25",  
    "JR東海道本線湯河原駅下車、タクシーで基本料金(徒歩20分)。厚木IC~小田原~真鶴道路→(49k)"  
),
(
    "ヒルトン東京お台場",  
    "17,358円~/人",  
    "東京都港区台場1-9-1",  
    "ゆりかもめ台場駅直結。りんかい線東京テレポート駅より徒歩約10分。羽田空港よりリムジンバスで約20分。"  
),
(
    "ホテルミュッセ銀座名鉄",  
    "9,500円~/人",  
    "東京都中央区銀座7丁目12番9号",  
    "JR、都営地下鉄浅草線「新橋」駅、東京メトロ「銀座」駅A3出口・「東銀座」駅A1出口より徒歩約6分"  
),
(
    "オークラアカデミアパークホテル",  
    "12,500円~/人",  
    "千葉県木更津市かずさ鎌足2-3-9",  
    "内房線木更津駅下車、バス20分。 木更津北IC出口を右折、かずさアカデミアパーク方面へ道なりに約15分。"  
),
(
    "アパホテル〈埼玉谷塚駅前〉",  
    "4,500円~/人",  
    "埼玉県草加市瀬崎1-7-20",  
    "■東武スカイツリーライン「谷塚駅」(東口)から徒歩約2分 ■首都高速6号三郷線「八潮南I.C」から車15分"  
),
(
    "京急EXイン 蒲田",  
    "4,700円~/人",  
    "東京都大田区蒲田5-28-18",  
    "京急蒲田駅徒歩5分/蒲田駅徒歩5分 【電車】京急線:羽田空港・品川駅から10分、JR線:東京駅25分"  
),
(
    "西鉄イン日本橋(人形町)",  
    "11,000円~/人",  
    "東京都中央区日本橋小舟町15-18",  
    "都営地下鉄浅草線・東京メトロ日比谷線「人形町駅」下車、A5出口から地上へ出て右方向へ徒歩約4分。"  
),
(
    "スーパーホテル東京・錦糸町駅前 高濃度人工炭酸泉 北斎の湯",  
    "10,800円~/人",  
    "東京都墨田区錦糸1-4-12",  
    "JR錦糸町駅北口下車徒歩3分、地下鉄半蔵門線錦糸町駅下車3番出口より徒歩4分"  
),
(
    "ホテルプレミアムグリーンソブリン",  
    "4,150円~/人",  
    "宮城県仙台市青葉区国分町2丁目9番22号",  
    "地下鉄勾当台公園駅【南3】徒歩4分 青葉通一番町駅【北1】徒歩10分 東北道仙台宮城IC【県庁方面】車15分"  
),
(
    "相鉄フレッサイン東京京橋",  
    "6,435円~/人",  
    "東京都中央区京橋2-11-1",  
    "JR東京駅八重洲中央口より徒歩8分。銀座線京橋駅6番出口・浅草線宝町駅A6番出口からいずれも徒歩約2分"  
),
(
    "東急ステイ銀座",  
    "11,600円~/人",  
    "東京都中央区銀座4-10-5",  
    "東京メトロ「銀座駅」A7出口から徒歩3分・東京メトロ、都営地下鉄「東銀座駅」A2出口から徒歩1分"  
),
(
    "ホテルインターゲート東京京橋",  
    "9,775円~/人",  
    "東京都中央区京橋3丁目7-8",  
    "東京メトロ銀座線「京橋駅」1番出口より徒歩1分、都営浅草線「宝町」A4出口より徒歩1分、東京駅徒歩約10分"  
),
(
    "相鉄フレッサイン東京東陽町駅前",  
    "3,930円~/人",  
    "東京都江東区東陽4-4-3",  
    "東京メトロ東西線東陽町駅5番出口に隣接、1番・4番出口より1分"  
),
(
    "メイプルイン幕張",  
    "4,200円~/人",  
    "千葉県千葉市花見川区幕張本郷1-12-1",  
    "幕張本郷駅(JR/京成)徒歩3分|幕張メッセ車10分・バス20分|TDR35分(車/電車)|東関道湾岸習志野ICより5分"  
),
(
    "レンブラントスタイル東京西葛西グランデ",  
    "5,225円~/人",  
    "東京都江戸川区西葛西6丁目19-18",  
    "東京メトロ東西線『西葛西駅』南口より徒歩4分。首都高速湾岸線葛西インターより10分"  
),
(
    "ホテルサンルート“ステラ”上野",  
    "6,400円~/人",  
    "東京都台東区上野7-7-1",  
    "JR上野駅浅草口より徒歩2分/東京メトロ銀座線・日比谷線浅草口より徒歩2分/京成上野駅より徒歩7分"  
),
(
    "相鉄フレッサイン新橋日比谷口",  
    "7,020円~/人",  
    "東京都港区新橋1-14-3",  
    "JR 新橋駅日比谷口より徒歩2分 東京メトロ銀座線 新橋駅7番出口より徒歩2分"  
),
(
    "ホテルグラフィー根津(HOTEL GRAPHY NEZU)",  
    "4,140円~/人",  
    "東京都台東区池之端4-5-10",  
    "千代田線「根津」駅より徒歩3分、南北線「東大前」駅より徒歩14分、JR「上野」駅より徒歩14分"  
),
(
    "芝パークホテル",  
    "15,225円~/人",  
    "東京都港区芝公園1-5-10",  
    "JR・東京モノレール 浜松町駅 徒歩8分、都営大江戸線浅草線 大門駅 徒歩4分、都営三田線 御成門駅 徒歩2分"  
),
(
    "LYURO 東京清澄 by THE SHARE HOTELS",  
    "4,100円~/人",  
    "東京都江東区清澄1-1-7",  
    "半蔵門線「水天宮前」駅徒歩10分 または 大江戸線「清澄白河」駅徒歩10 分"  
),
(
    "ホテルサンオーク",  
    "5,900円~/人",  
    "埼玉県越谷市南越谷1-22-1",  
    "JR武蔵野線南越谷駅下車徒歩3分、東武スカイツリーライン新越谷駅下車徒歩3分"  
),
(
    "【ホテル松風】庭園露天風呂と40種類の朝食バイキング",  
    "5,670円~/人",  
    "愛知県豊田市緑ケ丘5-11",  
    "東名高速豊田ICから車5分/トヨタ本社工場へ車5分/名鉄三河線土橋駅からタクシー5分"  
),
(
    "ホテルウィングインターナショナルセレクト浅草駒形",  
    "6,500円~/人",  
    "東京都台東区駒形2-7-5",  
    "【都営浅草線】浅草駅A2bエレベーター出口より徒歩0分 / 【東京メトロ銀座線】浅草駅4番出口より徒歩3分"  
),
(
    "スマイルホテル浅草",  
    "3,900円~/人",  
    "東京都台東区浅草6-35-8",  
    "東京メトロ銀座線「浅草駅」7番出口より徒歩約10~15分。都営バス東42甲「浅草七丁目」下車。"  
),
(
    "相鉄フレッサイン新橋烏森口(旧ホテルサンルート新橋)",  
    "6,210円~/人",  
    "東京都港区新橋4-10-2",  
    "JR新橋駅烏森出口より徒歩4分、銀座線都営浅草線新橋駅より徒歩6分、ゆりかもめ新橋駅より徒歩7分"  
),
(
    "ホテルリブマックスBUDGET東京木場",  
    "3,825円~/人",  
    "東京都江東区東陽1-27-6",  
    "東京メトロ東西線「木場駅」1番出口徒歩5分!「東陽町駅」2番出口徒歩6分!都営バス「東陽一丁目」徒歩2分"  
),
(
    "ホテルリソル秋葉原",  
    "7,200円~/人",  
    "東京都千代田区神田須田町2-25-12",  
    "JR線「秋葉原」駅中央改札出口より徒歩約3分、都営地下鉄新宿線「岩本町」駅A2出口より徒歩1分"  
),
(
    "後楽ガーデンホテル",  
    "11,400円~/人",  
    "東京都文京区後楽1-5-3",  
    "総武線「飯田橋」東口徒歩7分、地下鉄「飯田橋」A1出口徒歩7分、大江戸線「飯田橋」C3出口徒歩1分"  
),
(
    "リッチモンドホテル東京武蔵野",  
    "7,300円~/人",  
    "東京都武蔵野市中町2-4-1",  
    "東京から27分、新宿から14分。JR中央線、総武線、地下鉄東西線三鷹駅北口下車、徒歩7分。"  
),
(
    "変なホテル東京 浜松町",  
    "11,400円~/人",  
    "東京都港区浜松町1-24",  
    "徒歩約1~5分圏内で3駅5路線利用可、東京・品川・新宿・渋谷・横浜・羽田空港へも乗換不要でアクセス抜群!"  
),
(
    "hotel MONday Premium 豊洲",  
    "5,404円~/人",  
    "東京都江東区東雲1-2-11",  
    "有楽町線豊洲駅より徒歩7分。ホテル~豊洲駅間のシャトルバス運行。詳しいアクセスはHPをご覧下さい"  
),
(
    "東急ステイ新宿",  
    "11,430円~/人",  
    "東京都新宿区新宿3-7-1",  
    "新宿三丁目駅C2,C3,C4出口から徒歩1分、JR新宿駅東口、南口から徒歩8分"  
),
(
    "ホテルヒラリーズ赤坂",  
    "6,530円~/人",  
    "東京都港区赤坂3丁目12-5",  
    "東京メトロ千代田線「赤坂駅」1番出口より徒歩2分、東京メトロ銀座線・丸ノ内線「赤坂見附駅」より徒歩5分"  
),
(
    "ホテル21 cozy little hotel",  
    "4,500円~/人",  
    "滋賀県草津市若竹町7―10",  
    "車/名神栗東ICよりR1を草津方面へ約10分 新名神、草津田上ICより15分 電車/JR草津駅東口より徒歩7分"  
),
(
    "ホテルモントレ半蔵門",  
    "6,840円~/人",  
    "東京都千代田区一番町23-1",  
    "東京メトロ半蔵門線「半蔵門」駅5番出口目の前/東京メトロ有楽町線「麹町」駅~徒歩約7分/東京駅~約15分"  
),
(
    "東急ステイ門前仲町",  
    "6,500円~/人",  
    "東京都江東区富岡1-23-2",  
    "東京メトロ東西線 門前仲町駅 1番出口から徒歩1分、都営大江戸線 門前仲町駅 5番出口から徒歩5分"  
),
(
    "BAY HOTEL浦安駅前",  
    "5,500円~/人",  
    "千葉県浦安市北栄1-15-28 エクレ-ル2~10階",  
    "東京メトロ「浦安駅」より徒歩1分"  
),
(
    "東京ドームホテル",  
    "9,450円~/人",  
    "東京都文京区後楽1-3-61",  
    "徒歩1~6分で5路線自由自在。JR総武線.地下鉄三田線の水道橋駅+丸の内線.南北線の後楽園駅+大江戸線も可"  
),
(
    "ホテル ザ・マンハッタン",  
    "6,000円~/人",  
    "千葉県千葉市美浜区ひび野2-10-1",  
    "JR京葉線海浜幕張駅下車、南口より徒歩約4分。東関東自動車道湾岸習志野ICより約5分"  
),
(
    "スーパーホテルPremier池袋天然温泉 奥湯河原美肌の湯",  
    "5,510円~/人",  
    "東京都豊島区池袋2丁目64番6号",  
    "JR3線、東武線、西武線、地下鉄3線が乗り入れる池袋駅20A出口より徒歩約5分"  
),
(
    "結びの宿 愛隣館 【花巻温泉郷 里山料理と17の湯船が自慢】",  
    "13,200円~/人",  
    "岩手県花巻市鉛西鉛23",  
    "JR新花巻駅より40分、花巻駅より25分(無料送迎バス有)。東北道花巻南ICよりお車で20分。"  
),
(
    "リッチモンドホテル東京目白",  
    "4,050円~/人",  
    "東京都豊島区目白3-5-14",  
    "JR山手線目白駅を出て左手側へ徒歩2分!近隣に飲食店、コンビニ、薬局あり!周辺は閑静な住宅街・学校施設"  
),
(
    "ロッテシティホテル錦糸町",  
    "7,125円~/人",  
    "東京都墨田区錦糸4-6-1",  
    "東京駅からわずか9分。JR「錦糸町駅」北口より徒歩約1分。東京メトロ半蔵門線「錦糸町駅」5番出口直結!"  
),
(
    "クロス・ウェーブ船橋",  
    "5,060円~/人",  
    "千葉県船橋市本町2-9-3",  
    "京成船橋駅下車徒歩7分、東西線乗入れ・JR総武線船橋駅下車徒歩8分※詳しくはフォトギャラリーをご覧下さい"  
),
(
    "ホテルリソル池袋",  
    "6,200円~/人",  
    "東京都豊島区南池袋2-30-14",  
    "地下鉄有楽町線「東池袋駅」(1番出口)より徒歩1分 / JR「池袋駅」東口より徒歩6分"  
),
(
    "東急ステイ築地(旧東急ステイ東銀座)",  
    "8,460円~/人",  
    "東京都中央区築地4-11-5",  
    "東京メトロ日比谷線築地駅 1番出口から徒歩4分、日比谷線・都営地下鉄浅草線 東銀座駅6番出口から徒歩5分"  
),
(
    "ホテル イースト21東京 ~オークラホテルズ&リゾーツ~",  
    "6,816円~/人",  
    "東京都江東区東陽6-3-3",  
    "東京メトロ東西線「東陽町駅」1番出口より徒歩7分、 首都高速9号深川線木場出口より5分(箱崎方面のみ)"  
),
(
    "浦安ビューフォートホテル",  
    "5,148円~/人",  
    "千葉県浦安市猫実4-18-28",  
    "東京メトロ浦安駅より徒歩1分。TDR・舞浜駅へは、お車で約15分。都心へもアクセス便利。"  
),
(
    "東京グリーンパレス",  
    "6,205円~/人",  
    "東京都千代田区二番町2番地",  
    "有楽町線「麹町駅」5番出口より徒歩1分。他に半蔵門駅」「四ツ谷駅」「市ヶ谷駅」の4駅6路線が徒歩圏内"  
),
(
    "京成リッチモンドホテル東京押上",  
    "7,050円~/人",  
    "東京都墨田区押上1丁目8-23",  
    "各線【押上駅】 東京メトロ半蔵門線、京成押上線、都営浅草線 B3出口より徒歩1分"  
),
(
    "ホテルフランクス",  
    "3,550円~/人",  
    "千葉県千葉市美浜区ひび野2-10-2",  
    "海浜幕張駅徒歩4分|空港リムジンバスホテル目の前発着|幕張メッセ徒歩5分|舞浜駅まで乗換無25分|"  
),
(
    "スーパーホテル東京・亀戸",  
    "4,270円~/人",  
    "東京都江東区亀戸2-13-6",  
    "JR亀戸駅北口より徒歩約5分"  
),
(
    "豪華カプセルホテル 安心お宿プレミア新宿駅前店",  
    "5,980円~/人",  
    "東京都新宿区新宿4-2-10",  
    "新宿駅東南口より徒歩90秒!/バスタ新宿より徒歩90秒!/新宿3丁目駅徒歩5分"  
),
(
    "豪華カプセルホテル 安心お宿プレミア 新橋汐留店",  
    "4,980円~/人",  
    "東京都港区東新橋2-7-8",  
    "JR新橋駅より徒歩約240秒/汐留駅より徒歩約180秒/JR浜松町駅より徒歩約9分"  
),
(
    "Hope&Wish バケーションハウス 青と碧と白と沖縄",  
    "18,000円~/人",  
    "沖縄県国頭郡恩納村真栄田3537-2",  
    "沖縄県国頭郡恩納村真栄田3537-2 ※那覇空港から車で60分 ※路線バス・最寄りバス停から車で5分"  
),
(
    "ベッセルイン京成津田沼駅前",  
    "4,500円~/人",  
    "千葉県習志野市津田沼5丁目12番4号",  
    "京成本線「京成津田沼駅」南口から徒歩1分★JR「津田沼駅」からは徒歩約13分☆有料駐車場 Aパーク津田沼"  
),
(
    "ホテルサードニクス東京",  
    "9,500円~/人",  
    "東京都中央区八丁堀1-13-7",  
    "東京駅徒歩15分/メトロ八丁堀駅A4出口徒歩1分/JR八丁堀駅B1出口徒歩4分/メトロ茅場町駅1番出口徒歩4分"  
),
(
    "変なホテル舞浜東京ベイ",  
    "7,600円~/人",  
    "千葉県浦安市富士見5丁目3-20",  
    "東京ディズニーリゾート(R)最寄りのJR舞浜駅からシャトルバスで約5分!駐車場あり(要事前予約)"  
),
(
    "ホテルポートプラザちば",  
    "3,780円~/人",  
    "千葉県千葉市中央区千葉港8-5",  
    "京葉線「千葉みなと」駅より徒歩1分、電車で海浜幕張駅まで8分、舞浜駅まで25分。"  
),
(
    "ホテルシーサイド江戸川",  
    "7,000円~/人",  
    "東京都江戸川区臨海町6-2-2",  
    "JR京葉線東京駅より5駅13分、葛西臨海公園駅下車徒歩3分、葛西臨海公園内。"  
),
(
    "ベッセルイン八千代勝田台駅前",  
    "4,050円~/人",  
    "千葉県八千代市勝田台1-9-1",  
    "京成線・東葉高速鉄道勝田台駅より徒歩1分、大手町・ディズニーリゾートまで40分、成田空港まで30分圏内!"  
),
(
    "東横INN西船橋原木インター",  
    "4,568円~/人",  
    "千葉県市川市原木3-17-30",  
    "JR総武本線西船橋駅南口から徒歩15分"  
),
(
    "ワイヤーズホテル品川シーサイド",  
    "5,400円~/人",  
    "東京都品川区東大井1-9-37",  
    "りんかい線「品川シーサイド駅」より徒歩約5分 ・京浜急行「青物横丁駅」より徒歩約7分"  
),
(
    "R&Bホテル東日本橋",  
    "5,050円~/人",  
    "東京都中央区東日本橋2-15-6",  
    "地下鉄都営浅草線 東日本橋駅B3出口より徒歩約3分"  
),
(
    "ホテルリブマックス千葉蘇我駅前",  
    "3,188円~/人",  
    "千葉県千葉市中央区南町2-15-16",  
    "JR各線「蘇我」駅 徒歩 約1分"  
),
(
    "東京グランドホテル",  
    "5,380円~/人",  
    "東京都港区芝2-5-2",  
    "JR浜松町駅 徒歩10分 都営地下鉄三田線芝公園駅A1出口徒歩3分"  
),
(
    "ホテルマリナーズコート東京",  
    "6,000円~/人",  
    "東京都中央区晴海4-7-28",  
    "都営大江戸線勝どき駅A5出口より徒歩10分"  
),
(
    "徳寿司旅館",  
    "6,160円~/人",  
    "千葉県鎌ケ谷市東初富5-20-83",  
    "新京成線鎌ヶ谷大仏駅より徒歩5分。無料駐車場6台。"  
),
(
    "浦安サンホテル",  
    "5,900円~/人",  
    "千葉県浦安市当代島1-19-13",  
    "東京メトロ地下鉄東西線浦安駅より徒歩3分。TDRへは車で約15分、路線バスで約20分とアクセス良好。"  
),
(
    "ホテルリブマックス千葉みなと駅前",  
    "2,975円~/人",  
    "千葉県千葉市中央区中央港1-23-4",  
    "駅近!JR京葉線「千葉みなと」駅 西口 徒歩約1分 千葉都市モノレール「千葉みなと」駅 出口2 徒歩約2分"  
),
(
    "ダブルツリーbyヒルトン東京有明(2024年12月開業)",  
    "12,409円~/人",  
    "東京都江東区有明3-7-3",  
    "りんかい線「国際展示場駅」&ゆりかもめ「有明駅」隣接。東京ビッグサイトまで約4分"  
),
(
    "ホテルリブマックス千葉駅前",  
    "2,657円~/人",  
    "千葉県千葉市中央区新千葉2丁目3-2",  
    "JR総武本線、千葉都市モノレール「千葉駅・西口」より徒歩約2分"  
),
(
    "スマイルホテル東京西葛西",  
    "3,950円~/人",  
    "東京都江戸川区西葛西3丁目15番5号",  
    "東京メトロ東西線「西葛西駅」より徒歩約2分、ビジネスの中心地「大手町駅」までは乗換えなしで約20分!"  
),
(
    "ホテル トレンド 船橋",  
    "3,300円~/人",  
    "千葉県船橋市本町2-7-10",  
    "JR船橋駅下車徒歩10分、京成船橋駅下車徒歩8分。京葉道路・船橋出口より国道14号を千葉方面へ5分。"  
),
(
    "カナロア船橋",  
    "4,500円~/人",  
    "千葉県船橋市海神1-15-7-2",  
    "京成船橋駅より徒歩にて約9分、船橋駅より徒歩にて約11分。"  
),
(
    "ホテル ラ フィレンツェ",  
    "7,920円~/室",  
    "東京都足立区西保木間4-14-22",  
    "東武伊勢崎線「谷塚」駅から徒歩10分、又は、駅より東武バス 竹の塚行乗車3分、水神橋バス停下車真向い"  
),
(
    "ホテルテトラ鶴見",  
    "9,000円~/室",  
    "神奈川県横浜市鶴見区鶴見中央4-5-13",  
    "JR京浜東北線鶴見駅東口徒歩5分、京急鶴見駅東口徒歩3分 JR,市営地下鉄新横浜駅バス30分"  
),
(
    "ホテルリブマックス浅草スカイフロント",  
    "4,888円~/人",  
    "東京都墨田区業平1-10-4",  
    "都営地下鉄浅草線・京成線「押上」駅A2出口より徒歩3分、東京メトロ半蔵門線「押上」駅B2出口より徒歩4分"  
),
(
    "船橋シティホテル",  
    "6,800円~/人",  
    "千葉県船橋市本町1-20-13",  
    "JR総武線快速船橋駅下車、徒歩5分"  
),
(
    "シーサイドホテル九十九里",  
    "4,950円~/人",  
    "千葉県山武市本須賀3701",  
    "ちばフラワーバス海岸線成東海岸下車3分/山武成東IC→成東より約20分/特急フラワーライナー約70分"  
),
(
    "所沢第一ホテル",  
    "5,500円~/人",  
    "埼玉県所沢市金山町15-4",  
    "池袋・新宿から30分・横浜から60分・西武線西所沢駅から徒歩5分・関越道所沢ICより15分"  
),
(
    "ホテルリブマックスBUDGET横浜鶴見",  
    "3,187円~/人",  
    "神奈川県横浜市鶴見区鶴見中央1-21-13",  
    "JR横浜駅から京浜東北線で3駅のJR鶴見東口・京急鶴見駅から徒歩4分。首都高速横羽線汐入出口から車で10分"  
),


]

# SQLを実行
cur.executemany(sql_insert_many, hotels_data)

# コミット処理（データ操作を反映させる）
con.commit()

# DBへの接続を閉じる
con.close() 

### 分析

In [41]:
import sqlite3
import pandas as pd

# データベースファイルのパス
db_path = "tripdata.db"

# DBに接続
con = sqlite3.connect(path + db_name)

# 宿泊費の統計情報を取得するクエリ
query = """
SELECT 
    hotel_name, 
    CAST(REPLACE(REPLACE(price, '円～', ''), ',', '') AS INTEGER) AS price
FROM hotels;
"""

# クエリを実行してデータフレームに格納
df = pd.read_sql_query(query, con)

# 統計情報の計算
max_price = df["price"].max()  # 最大値
min_price = df["price"].min()  # 最小値
mean_price = df["price"].mean()  # 平均値
median_price = df["price"].median()  # 中央値

# 結果を出力
print(f"最大価格: {max_price}円")
print(f"最小価格: {min_price}円")
print(f"平均価格: {mean_price:.2f}円")
print(f"中央値: {median_price}円")

# DB接続を閉じる
con.close()


最大価格: 67850円
最小価格: 1円
平均価格: 7949.63円
中央値: 5660.0円


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 価格データを取得
query = """
SELECT 
    hotel_name, 
    CAST(REPLACE(REPLACE(price, '円～', ''), ',', '') AS INTEGER) AS price
FROM hotels;
"""
df = pd.read_sql_query(query, con)

# 価格データの統計情報を表示
print(df["price"].describe())

# グラフの作成
plt.figure(figsize=(12, 6))

# 1. ヒストグラム
plt.subplot(1, 3, 1)
sns.histplot(df["price"], kde=True, bins=15, color="skyblue")
plt.title("Price Distribution (Histogram)")
plt.xlabel("Price")
plt.ylabel("Frequency")

# グラフのレイアウトを調整して表示
plt.tight_layout()
plt.show()


In [ ]:
import sqlite3
from geopy.geocoders import Nominatim

db_path = "tripdata.db" 
con = sqlite3.connect(path + db_name)

# DBから住所を取得
query = "SELECT id, address FROM hotels;"  
data = con.execute(query).fetchall()

# Geolocatorを初期化
geolocator = Nominatim(user_agent="geoapiExercises")

# 緯度経度データを取得
location_data = []
for hotel_id, address in data:
    try:
        location = geolocator.geocode(address)
        if location:
            location_data.append((hotel_id, location.latitude, location.longitude))
        else:
            location_data.append((hotel_id, None, None))  # 見つからない場合
    except Exception as e:
        location_data.append((hotel_id, None, None))  # エラー時

# DBに緯度経度データを更新する準備
update_query = "UPDATE hotels SET latitude = ?, longitude = ? WHERE id = ?;"

# 緯度経度をDBに保存
for loc in location_data:
    con.execute(update_query, (loc[1], loc[2], loc[0]))

# コミットと接続終了
con.commit()
con.close()
